In [1]:
%pip install -q uv pyngrok
!uv pip install -q fastapi uvicorn open-clip-torch torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.2/21.2 MB 82.0 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
import open_clip

MODEL_ID = 'hf-hub:laion/CLIP-ViT-B-32-laion2B-s34B-b79K'
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

model, _, preprocess = open_clip.create_model_and_transforms(MODEL_ID, device=DEVICE)
model.eval()
model = model.to(torch.float16)
tokenizer = open_clip.get_tokenizer(MODEL_ID)

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

open_clip_config.json:   0%|          | 0.00/604 [00:00<?, ?B/s]

open_clip_model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

In [3]:
import torch
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
import open_clip


def l2_normalize(t: torch.Tensor) -> torch.Tensor:
    return torch.nn.functional.normalize(t, p=2, dim=-1)

class EmbedRequest(BaseModel):
    texts: list[str]
    normalize: bool = True

class EmbedResponse(BaseModel):
    embeddings: list[list[float]]

app = FastAPI()

@app.get("/")
def root():
    return {"status": "ok"}

@app.post('/embed', response_model=EmbedResponse)
def embed(req: EmbedRequest):
    tokens = tokenizer(req.texts).to(DEVICE)
    with torch.inference_mode():
        feats = model.encode_text(tokens)
        if req.normalize:
            feats = l2_normalize(feats)
        feats = feats.detach().float().cpu().numpy()
    return EmbedResponse(embeddings=feats.tolist())

In [4]:
import getpass
from pyngrok import ngrok, conf
import threading

print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
conf.get_default().auth_token  = user_secrets.get_secret("ngrok")


# Start uvicorn in background
config = uvicorn.Config(app, host='0.0.0.0', port=8000, log_level='info')
server = uvicorn.Server(config)
thread = threading.Thread(target=server.run, daemon=True)
thread.start()

# Expose via ngrok
tunnel = ngrok.connect(8000, "http")
print("Public URL:", tunnel.public_url)
print("POST", tunnel.public_url + "/embed")

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken


INFO:     Started server process [37]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://nonaggressive-elmer-acyclic.ngrok-free.dev                                      
POST https://nonaggressive-elmer-acyclic.ngrok-free.dev/embed


In [5]:
# import requests

# url = "https://nonaggressive-elmer-acyclic.ngrok-free.dev/embed"
# payload = {"texts": ["a photo of a dog", "a red car"], "normalize": True}
# r = requests.post(url, json=payload, timeout=15)
# r.raise_for_status()
# embeddings = r.json()["embeddings"]  # List[List[float]], shape [N, 512]
# print(len(embeddings), len(embeddings[0]))